# Lab 04: Ames Iowa Home Prices

---
author: Your Name Here
date: October 4, 2024
embed-resources: true
---

## Introduction

## Methods

In [50]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# machine learning
from sklearn.datasets import make_friedman1
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.metrics import accuracy_score, root_mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression

### Data

In [2]:
# load data
ames_train = pd.read_csv(
    "https://cs307.org/lab-04/data/ames-train.csv",
)
ames_test = pd.read_csv(
    "https://cs307.org/lab-04/data/ames-test.csv",
)

In [3]:
ames_train

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,2832,908188140,160,RM,24.0,2522,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2006,WD,Normal,137500
1,2736,905426150,80,RL,NaN,19690,Pave,NaN,IR1,Lvl,...,738,Gd,GdPrv,NaN,0,8,2006,WD,Alloca,274970
2,2135,907200130,20,RL,97.0,11800,Pave,NaN,IR1,Bnk,...,0,NaN,NaN,NaN,0,8,2007,WD,Family,131000
3,2424,528228415,120,RM,NaN,3072,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,5,2006,WD,Normal,178740
4,1967,535457020,20,RL,80.0,8000,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,11,2007,WD,Normal,156500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1870,1020,527302070,20,RL,NaN,10825,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,7,2008,WD,Normal,181900
1871,237,905426200,20,RL,65.0,11479,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,144500
1872,1547,910202050,30,RM,40.0,3636,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,1,2008,WD,Normal,55000
1873,1855,533251110,20,RL,80.0,12000,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,3,2007,WD,Normal,255000


In [6]:
# summary statistics
len(ames_train), ames_train.shape[1] - 1

(1875, 81)

In [ ]:
# visualizations

### Models

In [7]:
# process data for ML
# create X and y for train dataset
X_train = ames_train.drop("SalePrice", axis=1)
y_train = ames_train["SalePrice"]

# create X and y for test dataset
X_test = ames_test.drop("SalePrice", axis=1)
y_test = ames_test["SalePrice"]

In [45]:
# helper function to find columns that are mostly missing
def columns_missing_above_threshold(df, threshold):
    missing_proportion = df.isnull().sum() / len(df)
    columns_above_threshold = missing_proportion[missing_proportion > threshold].index.tolist()
    return columns_above_threshold

# define columns to be excluded
many_missing_columns = columns_missing_above_threshold(X_train, 0.6)
exclude_columns = ["Order", "PID"] + many_missing_columns

# find columns with string data type (will be considered categorical)
string_columns = X_train.select_dtypes(include=["object"]).columns.tolist()
string_columns = [col for col in string_columns if col not in exclude_columns]

# find columns with numeric data type
numeric_columns = X_train.select_dtypes(include=["int64", "float64"]).columns.tolist()
numeric_columns = [col for col in numeric_columns if col not in exclude_columns]

In [43]:
# categorical_features = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
# numeric_features = X_train.select_dtypes(include=['int', 'float']).columns.tolist()

# len(categorical_features) + len(numeric_features)


81

In [55]:
# train models
# pipeline for numeric
numeric_preprocessor = Pipeline(
    steps=[
        ("MedianImputer", SimpleImputer(strategy="median")),
        ("Standardize", StandardScaler()),
    ]
)

# pipeline for categorical
categorical_preprocessor = Pipeline(
    steps=[
        ("ModalImputer", SimpleImputer(strategy="most_frequent")),
        ("OneHotEncoder", OneHotEncoder(handle_unknown="infrequent_if_exist", max_categories=5)),
    ]
)

# column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ("ExcludeColumns", "drop", exclude_columns),
        ("CategoricalPreprocessing", categorical_preprocessor, string_columns),
        ("NumericProcessing", numeric_preprocessor, numeric_columns),
    ],
    remainder="drop",
)


pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("Regressor", DummyRegressor()),
    ]
)

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import HistGradientBoostingRegressor


param_grid = {
    "Regressor": [HistGradientBoostingRegressor()],
    "Regressor__learning_rate": [0.1, 0.01, 0.001],
    "Regressor__max_iter": [1000],
    "Regressor__max_depth": [None, 3],
    "Regressor__l2_regularization": [0.1, 1.0]
}


grid_search = mod = GridSearchCV(pipeline, param_grid=param_grid, n_jobs=-1, cv=5, verbose=2, scoring="neg_mean_absolute_percentage_error",)
grid_search.fit(X_train, y_train)

print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation MAPE: {grid_search.best_score_}")


Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best parameters: {'Regressor': HistGradientBoostingRegressor(), 'Regressor__l2_regularization': 1.0, 'Regressor__learning_rate': 0.01, 'Regressor__max_depth': None, 'Regressor__max_iter': 1000}
Best cross-validation MAPE: -0.09323748138117231


## Results

In [56]:
# report model metrics
from sklearn.metrics import mean_absolute_percentage_error
y_pred = grid_search.predict(X_test)
mean_absolute_percentage_error(y_test, y_pred)

0.08164649164771304

In [57]:
from joblib import dump

dump(grid_search, "ames-housing.joblib")

['ames-housing.joblib']

## Discussion

### Conclusion